##### Ler arquivos desbloqueados/unlocked da landing layer

01. Utilizar spark.read() para leitura do .csv
02. Renomear colunas para snake_case e remover lixo
03. Filtrar com regex as datas
04. Adicionar ingestion_timestamp e _metadadata.file_path
05. Salvar como tabela personalfinance.bronze.gastos_btg

In [0]:
%run "./support"

Passo 01 - Utilizar spark.read() para leitura do .csv

In [0]:
df = spark.read \
    .format('csv') \
    .option('header', True) \
    .option('inferSchema', False) \
    .load('/Volumes/personalfinance/landing/vol_landing/btg/unlocked/*/*.csv')

In [0]:
colunas_to_rename = {
 'Unnamed: 1': 'data_da_compra',
 'Unnamed: 2': 'descricao',
 'Unnamed: 4': 'valor_brl',
 'Unnamed: 5': 'tipo_compra',
 'Unnamed: 6': 'id_code',
 'Unnamed: 7': 'card_final'
}

Passo 02 - Renomear colunas para snake_case e remover lixo

In [0]:
df = df.withColumnsRenamed(colunas_to_rename) \
    .select(*colunas_to_rename.values())

Passo 03 - Filtrar com regex as datas

In [0]:
df = df.filter(df.data_da_compra.rlike(r'^\d{4}-\d{2}-\d{2}'))

Passo 04 - Adicionar ingestion_timestamp e _metadadata.file_name

In [0]:
df = add_ingestion_timestamp(df)
df = add_file_metadata(df)

Passo 05 - Salvar como tabela personalfinance.bronze.gastos_btg

In [0]:
display(df.limit(10))

In [0]:
df.write \
  .mode('overwrite') \
  .format('delta') \
  .option('overwriteSchema', True) \
  .saveAsTable('personalfinance.bronze.gastos_btg')